In [1]:
import os
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [2]:
DATASET_PATH = r"/Users/ayushagrawal/Desktop/dhriti/train"

img_size = 128
images = []
labels = []

# Get class folders
class_names = [d for d in os.listdir(DATASET_PATH) if os.path.isdir(os.path.join(DATASET_PATH, d))]
class_names.sort()  # stable ordering

print("Detected classes:", class_names)

for label_index, class_name in enumerate(class_names):
    class_path = os.path.join(DATASET_PATH, class_name)

    # Loop all files inside class folder
    for img_file in os.listdir(class_path):

        img_path = os.path.join(class_path, img_file)
        if not img_file.lower().endswith((".jpg", ".jpeg", ".png", ".bmp")):
            continue

        img = cv2.imread(img_path)

        if img is None:
            print("Failed to load:", img_path)
            continue

        img = cv2.resize(img, (img_size, img_size))

        images.append(img)
        labels.append(label_index)




Detected classes: ['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_Leaf_Mold', 'Tomato_Septoria_leaf_spot', 'Tomato__Target_Spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Tomato__Tomato_mosaic_virus', 'Tomato_healthy']


In [4]:
# X = np.array(images) / 255.0
# y = to_categorical(np.array(labels), num_classes=len(class_names))

# print("Loaded images:", X.shape)
# print("Loaded labels:", y.shape)

# # Train-test split
X = np.array(images) / 255.0
y = to_categorical(np.array(labels), num_classes=len(class_names))

print("Total images:", len(X))
print("Total classes:", len(class_names))
print("X shape:", X.shape)
print("y shape:", y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Total images: 13264
Total classes: 14
X shape: (13264, 128, 128, 3)
y shape: (13264, 14)


In [6]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(img_size, img_size, 3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dense(len(class_names), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=32)

# Save Model
model.save("crop_disease_manual_model.h5")

print("Model saved as crop_disease_manual_model.h5")


Epoch 1/5
332/332 ━━━━━━━━━━━━━━━━━━━━ 49s 147ms/step - accuracy: 0.5883 - loss: 1.2800 - val_accuracy: 0.7580 - val_loss: 0.7453
Epoch 2/5
332/332 ━━━━━━━━━━━━━━━━━━━━ 41s 124ms/step - accuracy: 0.8400 - loss: 0.4929 - val_accuracy: 0.7942 - val_loss: 0.6211
Epoch 3/5
332/332 ━━━━━━━━━━━━━━━━━━━━ 43s 128ms/step - accuracy: 0.9093 - loss: 0.2752 - val_accuracy: 0.8243 - val_loss: 0.5264
Epoch 4/5
332/332 ━━━━━━━━━━━━━━━━━━━━ 43s 130ms/step - accuracy: 0.9452 - loss: 0.1705 - val_accuracy: 0.8643 - val_loss: 0.4569
Epoch 5/5
332/332 ━━━━━━━━━━━━━━━━━━━━ 42s 125ms/step - accuracy: 0.9756 - loss: 0.0800 - val_accuracy: 0.8504 - val_loss: 0.5155


Model saved as crop_disease_manual_model.h5
